# Project-2
## Tanvir Khan, Nicky Pant, Paul Pineda, James Ye, Fabienne Zumbuehl
### Load CSV Files into Postgres database (StocksDataBase)

In [1]:
import pandas as pd
#from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from sqlalchemy import create_engine
import psycopg2
import glob

In [2]:
# Creating connection with database
# StocksDataBase is database name for this project
connection_string = "postgres:postgres@localhost:5432/StocksDataBase"
engine = create_engine(f'postgresql://{connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['company', 'price']

# Load company.xlsx file

In [4]:
company_df = pd.read_excel("data/company.xlsx");
company_df.head()

,ranking,name,ticker,mkt_cap,pe_ratio,eps,dividend_pct,exchange,esg_score,recom_rating,sector,industry,country,city,latitude,longitude
0,1,Walmart,WMT,339.211,22.78,5.26,1.81,NYSE,29.0,1.9,Consumer Defensive,Discount Stores,USA,Bentonville,36.365265,-94.219827
1,2,Sinopec Group,SHI,4.377,2.54,9.40,7.08,NYSE,NaN,4.0,Energy,Oil & Gas Refining & Marketing,China,Peking,30.711817,121.329752
2,3,Royal Dutch Shell,RDS.A,121.519,13.36,2.45,3.83,NYSE,NaN,2.4,Energy,Oil & Gas Integrated,Netherlands,The Hague,52.093801,4.310453
3,4,China National Petroleum,SNP,62.697,4.61,9.10,10.27,NYSE,NaN,2.0,Energy,Oil & Gas Integrated,China,Peking,39.957500,115.937337
4,6,Saudi Aramco,2222.SR,7146.000,20.90,1.55,1.19,Tadawul,NaN,NaN,Energy,Oil & Gas Integrated,Saudi Arabia,Dhahran,26.227210,49.482172


In [5]:
# rerange column position to match database table columns
company_df = company_df[["ticker","name", "ranking", "mkt_cap", "pe_ratio", "eps", "dividend_pct", "exchange", "esg_score", "recom_rating", "sector", "industry", "country", "city", "latitude", "longitude"]]
company_df.head()

,ticker,name,ranking,mkt_cap,pe_ratio,eps,dividend_pct,exchange,esg_score,recom_rating,sector,industry,country,city,latitude,longitude
0,WMT,Walmart,1,339.211,22.78,5.26,1.81,NYSE,29.0,1.9,Consumer Defensive,Discount Stores,USA,Bentonville,36.365265,-94.219827
1,SHI,Sinopec Group,2,4.377,2.54,9.40,7.08,NYSE,NaN,4.0,Energy,Oil & Gas Refining & Marketing,China,Peking,30.711817,121.329752
2,RDS.A,Royal Dutch Shell,3,121.519,13.36,2.45,3.83,NYSE,NaN,2.4,Energy,Oil & Gas Integrated,Netherlands,The Hague,52.093801,4.310453
3,SNP,China National Petroleum,4,62.697,4.61,9.10,10.27,NYSE,NaN,2.0,Energy,Oil & Gas Integrated,China,Peking,39.957500,115.937337
4,2222.SR,Saudi Aramco,6,7146.000,20.90,1.55,1.19,Tadawul,NaN,NaN,Energy,Oil & Gas Integrated,Saudi Arabia,Dhahran,26.227210,49.482172


In [6]:
# remove old company data
engine.execute("delete from company")

# load company dataframe into database
company_df.to_sql(name='company', con=engine, if_exists='append', chunksize = 20, index=False)
company_df2 = pd.read_sql("select * from company", con=engine)
company_df2.count()

ticker          172
name            172
ranking         172
mkt_cap         171
pe_ratio        144
eps             126
dividend_pct    142
exchange        171
esg_score       101
recom_rating    115
sector          171
industry        171
country         171
city            171
latitude        171
longitude       171
dtype: int64

## Load price csv files

In [7]:
priceCSVFiles = glob.glob("data/*.csv")
priceCSVFiles

['data\\066570.KS.csv',
 'data\\600104.SS.csv',
 'data\\601100.SS.csv',
 'data\\ACGBY.csv',
 'data\\ALIZF.csv',
 'data\\AMX.csv',
 'data\\AXAHF.csv',
 'data\\BABA.csv',
 'data\\BAC.csv',
 'data\\BACHY.csv',
 'data\\BAM.csv',
 'data\\BMWYY.csv',
 'data\\BUD.csv',
 'data\\CAH.csv',
 'data\\CAT.csv',
 'data\\CHDRF.csv',
 'data\\CHL.csv',
 'data\\CHPXF.csv',
 'data\\CICHF.csv',
 'data\\CIHKY.csv',
 'data\\COST.csv',
 'data\\CRWOY.csv',
 'data\\CWYCF.csv',
 'data\\FNMA.csv',
 'data\\GE.csv',
 'data\\GM.csv',
 'data\\GOOG.csv',
 'data\\HMC.csv',
 'data\\ITUB.csv',
 'data\\JPHLF.csv',
 'data\\JPM.csv',
 'data\\KEP.csv',
 'data\\KR.csv',
 'data\\LFC.csv',
 'data\\LMT.csv',
 'data\\LUKOY.csv',
 'data\\MSBHY.csv',
 'data\\MSFT.csv',
 'data\\OGZPY.csv',
 'data\\PFE.csv',
 'data\\PSX.csv',
 'data\\RWEOY.csv',
 'data\\SCGLY.csv',
 'data\\VLO.csv',
 'data\\VZ.csv',
 'data\\WBA.csv',
 'data\\WFAFY.csv']

In [8]:
# remove old price data
engine.execute("delete from price")

# loop through all data/*.csv files and load them into price table
for file in priceCSVFiles:
    file = file.strip()
    
    print(file)
    price_df = pd.read_csv(file)
    print(price_df.head())
    csv_file = file.split('\\')[1]
    
    # ignore company.csv, we will handle it differetly
    if csv_file == "company.csv":
        continue

    ticker = csv_file.strip('.csv')
    price_df["ticker"] = ticker
    print(ticker)
    
    # rename column heading to match database table columns
    price_df.rename(columns = {"Date":"date", "Open":"open", "High":"high", "Low":"low", "Close":"close", "Adj Close":"adj_close", "Volume":"volume"}, inplace=True)
    # make ticker as the first column
    price_df = price_df[["ticker", "date", "open", "high", "low", "close", "adj_close", "volume"]]
    
    # load dataframe into database
    try:
        price_df.to_sql(name='price', con=engine, if_exists='append', chunksize = 20, index=False)
    except Exception as e:
        print(e)

# check out the price table
price_df2 = pd.read_sql("select * from price", con=engine)
price_df2.count()

data\066570.KS.csv
         Date     Open     High      Low    Close     Adj Close    Volume
0  2015-07-01  47200.0  48150.0  46900.0  47800.0  46297.730469  680372.0
1  2015-07-02  48200.0  48600.0  47900.0  48100.0  46588.300781  722630.0
2  2015-07-03  48250.0  48450.0  47400.0  47650.0  46152.437500  521621.0
3  2015-07-06  46750.0  46850.0  46000.0  46050.0  44602.730469  899759.0
4  2015-07-07  46350.0  46550.0  45700.0  46550.0  45087.015625  888051.0
066570.KS
data\600104.SS.csv
         Date       Open       High        Low      Close  Adj Close  \
0  2015-07-01  22.100000  22.580000  21.450001  21.620001  15.566809   
1  2015-07-02  21.790001  22.110001  20.600000  21.150000  15.228400   
2  2015-07-03  21.250000  21.510000  19.580000  20.260000  14.587584   
3  2015-07-06  22.250000  22.260000  21.320000  21.900000  15.768413   
4  2015-07-07  21.790001  23.139999  21.420000  23.080000  16.618038   

      Volume  
0   91470020  
1   90836062  
2   98059183  
3  226156959  


data\CICHF.csv
         Date  Open  High   Low  Close  Adj Close  Volume
0  2015-07-01  0.90  0.90  0.90   0.90   0.467748    1000
1  2015-07-02  0.90  0.90  0.90   0.90   0.467748       0
2  2015-07-06  0.87  0.88  0.87   0.88   0.457354    2500
3  2015-07-07  0.87  0.88  0.86   0.86   0.446959  598300
4  2015-07-08  0.83  0.83  0.83   0.83   0.431368   15500
CICHF
data\CIHKY.csv
         Date   Open   High    Low  Close  Adj Close  Volume
0  2015-07-01  14.48  14.68  14.24  14.68  12.174124    3200
1  2015-07-02  14.63  14.63  14.51  14.51  12.033143    1500
2  2015-07-06  13.97  14.19  13.90  13.90  11.527269    4200
3  2015-07-07  13.29  13.34  13.00  13.34  11.062862    9900
4  2015-07-08  12.43  12.58  12.05  12.05   9.993065    9200
CIHKY
data\COST.csv
         Date        Open        High         Low       Close   Adj Close  \
0  2015-07-01  135.789993  136.570007  135.539993  136.490005  124.404877   
1  2015-07-02  137.000000  137.199997  136.279999  136.389999  124.313690   

data\MSBHY.csv
         Date       Open       High        Low      Close  Adj Close  Volume
0  2015-07-01  43.990002  44.160000  43.990002  44.099998  37.628399   40800
1  2015-07-02  44.330002  44.450001  44.330002  44.450001  37.927036    1300
2  2015-07-06  44.450001  44.450001  44.450001  44.450001  37.927036       0
3  2015-07-07  43.599998  43.790001  43.599998  43.790001  37.363888    2100
4  2015-07-08  41.799999  42.360001  41.799999  42.060001  35.887764    4200
MSBHY
data\MSFT.csv
         Date       Open       High        Low      Close  Adj Close    Volume
0  2015-07-01  44.459999  45.230000  44.099998  44.450001  40.228317  28343900
1  2015-07-02  44.480000  44.750000  44.060001  44.400002  40.183064  21752000
2  2015-07-06  43.959999  44.480000  43.950001  44.389999  40.174019  23034000
3  2015-07-07  44.340000  44.490002  43.320000  44.299999  40.092571  36435800
4  2015-07-08  44.439999  44.900002  44.029999  44.240002  40.038265  39785900
MSFT
data\OGZPY.csv
         

ticker       58940
date         58940
open         58934
high         58934
low          58934
close        58934
adj_close    58934
volume       58934
dtype: int64